## Prediction model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
salary_df = pd.read_csv("dataset_for_prediction.csv")
salary_df.head()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,MP,PER,TS%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Salaries,Rank
0,0,2000,Ray Allen,SG,24.0,MIL,82.0,3070.0,20.6,0.570,...,276.0,359.0,308.0,110.0,19.0,183.0,187.0,1809.0,9000000.0,14
1,1,2000,Rafer Alston,PG,23.0,MIL,27.0,361.0,4.3,0.310,...,18.0,23.0,70.0,12.0,0.0,29.0,29.0,60.0,301000.0,144
2,2,2000,Kenny Anderson,PG,29.0,BOS,82.0,2593.0,17.4,0.524,...,170.0,225.0,420.0,139.0,8.0,130.0,230.0,1149.0,6680000.0,30
3,3,2000,Darrell Armstrong,PG,31.0,ORL,82.0,2590.0,19.5,0.542,...,205.0,270.0,501.0,169.0,9.0,248.0,137.0,1330.0,4125000.0,56
4,4,2000,Isaac Austin,C,30.0,WAS,59.0,1173.0,10.2,0.481,...,218.0,282.0,74.0,17.0,38.0,107.0,128.0,397.0,4819000.0,44


In [4]:


data_dum = pd.get_dummies(columns=["Pos", "Tm"], data=salary_df)
prediction_data = data_dum.drop(columns = ["Player"])
print(prediction_data)

       Unnamed: 0  Year   Age     G      MP   PER    TS%   3PAr    FTr  ORB%  \
0               0  2000  24.0  82.0  3070.0  20.6  0.570  0.288  0.282   3.2   
1               1  2000  23.0  27.0   361.0   4.3  0.310  0.147  0.042   1.6   
2               2  2000  29.0  82.0  2593.0  17.4  0.524  0.223  0.257   2.3   
3               3  2000  31.0  82.0  2590.0  19.5  0.542  0.360  0.221   2.7   
4               4  2000  30.0  59.0  1173.0  10.2  0.481  0.011  0.389   6.2   
5               5  2000  28.0  80.0  2726.0  14.6  0.609  0.564  0.222   2.0   
6               6  2000  19.0  24.0   130.0   7.6  0.397  0.171  0.343   3.6   
7               7  2000  27.0  82.0  1691.0  16.5  0.569  0.166  0.339   1.1   
8               8  2000  21.0  82.0  3155.0  16.7  0.521  0.206  0.242   2.8   
9               9  2000  32.0  73.0  2459.0  15.1  0.468  0.360  0.114   2.2   
10             10  2000  22.0  46.0   423.0  10.0  0.463  0.009  0.330   5.7   
11             11  2000  20.0  81.0  299

In [5]:
from sklearn.model_selection import train_test_split
X = prediction_data.copy().drop(columns=["Salaries"])
y = prediction_data["Salaries"]

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

### Predicting using Random Forest ###

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# TODO
rf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0).fit(X_train, y_train)

In [7]:
rayAllen = data_dum[data_dum["Player"] == "Kobe Bryant"]
rayAllen = rayAllen.drop(columns=["Player", "Salaries"])
rayAllen_pred = rf.predict(rayAllen)

### Predicting using Linear Regression ###

In [10]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
print(lr.coef_)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [29]:
print(lr.predict(rayAllen[rayAllen["Year"] == 2005]))
print(salary_df[salary_df["Player"] == "Kobe Bryant"]["Salaries"][salary_df["Year"] == 2005])

[12687058.78066832]
2114    14175000.0
Name: Salaries, dtype: float64


### Predicting Key Players Salaries ###
#### Predicting new contract salary by year ####
Chosen players:
* Anthony Davis
* Damian Lillard
* Giannis Antetokounmpo

In [48]:
players = ["Anthony Davis", "Damian Lillard", "Giannis Antetokounmpo"]
choosenModel = lr
for player in players:
    predictedSalary = choosenModel.predict(data_dum[(data_dum["Player"] == player) & (salary_df["Year"] == 2019 )].drop(columns=["Player", "Salaries"]))
    salaryNow = salary_df[(salary_df["Player"] == player) & (salary_df["Year"] == 2019 )]["Salaries"].iloc[0]
    print("Player: " + str(player)+"; "+ "Current Salary: "+ str(salaryNow) +"; "+"Predicted next season salary: "+ str(predictedSalary))
    



Player: Anthony Davis; Current Salary: 25434263.0; Predicted next season salary: [19987990.6008805]
Player: Damian Lillard; Current Salary: 27977689.0; Predicted next season salary: [19403331.80264282]
Player: Giannis Antetokounmpo; Current Salary: 24157304.0; Predicted next season salary: [22772073.40977889]
